In [24]:
import numpy as np
import pandas as pd

import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy

import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import random

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from src import nlp_utils
from src.process_text_variables import contracted_words_dict, stop_words_dict, punc, stop_words_incl_in_sentiment_dict 
from collections import Counter

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 1) Import and Process Data

In [3]:
wsm = pd.read_csv('data/wallstreet_master.csv')
stop_words_dict = stop_words_dict + stop_words_incl_in_sentiment_dict
wsm['tokens'] = wsm['contentWithHTMLTag'].apply(lambda x: nlp_utils.process_text(x, contracted_words_dict, punc, stop_words_dict, min_len=2))

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
train_split_idx = 41000

all_docs_tokenized = wsm['tokens'].tolist()
# create copy to have record of original order

all_docs_randomized = all_docs_tokenized[:]
random.shuffle(all_docs_randomized)
train_docs_tokenized = all_docs_randomized[:train_split_idx]
test_docs_tokenized = all_docs_randomized[train_split_idx]

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 2) Create Bigrams/Trigrams and Remove Neglible Words Based on Parts of Sentence (PoS)

In [5]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# English multi-task CNN trained on OntoNotes. Assigns context-specific token vectors, POS tags, dependency parse and named entities.
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADJ', 'PROPN']):
    texts_revised =[]
    for post in texts:
        doc = nlp(' '.join(post))
        texts_revised.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_revised

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Function Used to Calculate Score that Corresponds to "threshold" hyperparameter**

```
def original_scorer(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count):
   #...
   """
    worda_count : int
        Number of occurrences for first word.
    wordb_count : int
        Number of occurrences for second word.
    bigram_count : int
        Number of co-occurrences for phrase "worda_wordb".
    len_vocab : int
        Size of vocabulary.
    min_count: int
        Minimum collocation count threshold.
    corpus_word_count : int
        Not used in this particular scoring technique.
    """
    #...

    return (bigram_count - min_count) / worda_count / wordb_count * len_vocab
  ```

In [6]:
min_count = 10
# greater the threshold, the lower the number of words.
threshold = 50


bigram = gensim.models.Phrases(all_docs_tokenized, min_count=min_count, threshold=threshold)
trigram = gensim.models.Phrases(bigram[all_docs_tokenized], threshold=threshold)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
docs_incl_bigrams = make_bigrams(all_docs_tokenized)

docs_lemmatized = lemmatization(docs_incl_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADJ', 'PROPN'])

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
docs_lemmatized[22222]

## 3) Create Corpus

In [8]:
# Bag of Words corpus
id2word = corpora.Dictionary(docs_lemmatized)

# Create unique ID for each word in corpus.
bow_corpus = [id2word.doc2bow(post) for post in  docs_lemmatized]

# Use the unique id as the index in id2word to see the corresponding token.
print(id2word[3])

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


go


In [9]:
# Tfidf corpus
tfidf = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 4) Develop LDA Model

In [10]:
# BoW 
lda_model_bow = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                               id2word=id2word,
                                               num_topics=20, 
                                               random_state=3,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# Tf-Idf
lda_model_tfidf = gensim.models.ldamodel.LdaModel(corpus=tfidf_corpus,
                                               id2word=id2word,
                                               num_topics=20, 
                                               random_state=3,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
print('BoW TOPICS:')
for t in lda_model_bow.print_topics(): 
    print(t)
    print('\n')

BoW TOPICS:
(0, '0.155*"cash" + 0.100*"custom" + 0.078*"eu" + 0.074*"access" + 0.073*"setup" + 0.049*"release" + 0.038*"tutorial" + 0.035*"multiple" + 0.030*"worry" + 0.029*"bin"')


(1, '0.306*"bank" + 0.225*"account" + 0.127*"credit" + 0.067*"fullz" + 0.040*"cc" + 0.035*"paypal" + 0.032*"fraud" + 0.027*"login" + 0.011*"serious" + 0.011*"asap"')


(2, '0.219*"allow" + 0.117*"external_contact" + 0.042*"passport" + 0.019*"hologram" + 0.000*"_" + 0.000*"phished_rule" + 0.000*"delivery_dot" + 0.000*"transfer" + 0.000*"fake" + 0.000*"real"')


(3, '0.416*"guide" + 0.054*"nice" + 0.049*"place" + 0.046*"interested" + 0.040*"s" + 0.030*"drug" + 0.029*"learn" + 0.024*"word" + 0.023*"opsec" + 0.022*"safe"')


(4, '0.188*"note" + 0.088*"sign" + 0.048*"euro" + 0.040*"pas" + 0.035*"cannabis" + 0.034*"print" + 0.031*"normal" + 0.025*"grade" + 0.024*"water" + 0.021*"like"')


(5, '0.151*"name" + 0.062*"escrow" + 0.057*"cheap" + 0.038*"scan" + 0.035*"topic" + 0.032*"join" + 0.030*"worth" + 0.028*"lar

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
print('Tf-Idf TOPICS:')
for t in lda_model_tfidf.print_topics(): 
    print(t)
    print('\n')

Tf-Idf TOPICS:
(0, '0.039*"put" + 0.033*"problem" + 0.033*"allow" + 0.027*"eu" + 0.026*"kind" + 0.023*"setup" + 0.023*"thing" + 0.016*"external_contact" + 0.016*"item" + 0.011*"tutorial"')


(1, '0.069*"cc" + 0.035*"asap" + 0.023*"serious" + 0.016*"canada" + 0.000*"bump" + 0.000*"_" + 0.000*"record_cashout" + 0.000*"phished_rule" + 0.000*"room" + 0.000*"everythingcc_bank"')


(2, '0.021*"atm" + 0.011*"moment" + 0.010*"solution" + 0.000*"bump" + 0.000*"_" + 0.000*"record_cashout" + 0.000*"phished_rule" + 0.000*"room" + 0.000*"everythingcc_bank" + 0.000*"picsou"')


(3, '0.058*"profile" + 0.055*"guide" + 0.053*"order" + 0.041*"write" + 0.036*"pm" + 0.032*"message" + 0.026*"go" + 0.025*"buy" + 0.022*"day" + 0.021*"thank"')


(4, '0.193*"bank" + 0.088*"drop" + 0.035*"cash" + 0.028*"end" + 0.012*"e" + 0.012*"pas" + 0.011*"depend" + 0.010*"trouble" + 0.008*"wall_street" + 0.008*"decrypt"')


(5, '0.030*"sign" + 0.030*"scan" + 0.026*"cheap" + 0.026*"join" + 0.022*"topic" + 0.017*"worth" + 0.0

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 4) Model Performance Evaluation - Baseline

### Bag-of-Words

In [15]:
# The lower, the better.
print('Perplexity: ', lda_model_bow.log_perplexity(bow_corpus))

coherence_model_bow = CoherenceModel(model=lda_model_bow, texts=docs_lemmatized, dictionary=id2word, coherence='c_v')
coherence_bow = coherence_model_bow.get_coherence()
print('Coherence Score: ', coherence_bow)

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Perplexity:  -17.098545791997772
Coherence Score:  0.35361736822624434


### Tf-Idf

In [16]:
print('Perplexity: ', lda_model_tfidf.log_perplexity(tfidf_corpus))

coherence_model_tfidf = CoherenceModel(model=lda_model_tfidf, texts=docs_lemmatized, dictionary=id2word, coherence='c_v')
coherence_tfidf = coherence_model_tfidf.get_coherence()
print('Coherence Score: ', coherence_tfidf)

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Perplexity:  -19.951403084908748
Coherence Score:  0.31366446152744454


# 5) Visualize Topics-Keywords

In [28]:
# Bag-of-Words
pyLDAvis.gensim.prepare(lda_model_bow, bow_corpus, id2word)

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.314653 -0.353913       1        1  33.974957
15     0.306466  0.217339       2        1  21.667558
13     0.115023 -0.026490       3        1   5.676676
17     0.213303  0.242253       4        1   5.493731
1      0.119457 -0.027460       5        1   5.102562
6      0.039749 -0.010619       6        1   3.469302
3      0.049776 -0.011995       7        1   3.193976
16     0.004544 -0.006810       8        1   2.984813
14    -0.034424 -0.007692       9        1   2.930648
5     -0.028678 -0.004341      10        1   2.469064
7     -0.011807 -0.005456      11        1   2.415312
19    -0.038387 -0.003727      12        1   1.948603
10    -0.056793 -0.002737      13        1   1.886010
12    -0.090029 -0.001386      14        1   1.568199
4     -0.106798  0.002694      15        1   1.324458
0     -0.108139 -0.000816      16        1   1.291685
8     -0.138493 -0.000096      17        1   1.165737
2     -0.185305  0.000497      18        1   0.672214
18    -0.160489  0.000265      19        1   0.639654
9     -0.203629  0.000490      20        1   0.124844, topic_info=                 Term          Freq         Total Category  logprob  loglift
172             offer  20481.000000  20481.000000  Default   30.000  30.0000
924              bank  20787.000000  20787.000000  Default   29.000  29.0000
92              guide  17698.000000  17698.000000  Default   28.000  28.0000
67              order  19221.000000  19221.000000  Default   27.000  27.0000
0                card  15041.000000  15041.000000  Default   26.000  26.0000
...               ...           ...           ...      ...      ...      ...
14853        expectus      0.022692      1.454591  Topic20  -11.201   2.5254
19500       chat_room      0.022692      1.454396  Topic20  -11.201   2.5255
14866  record_cashout      0.022692      1.455305  Topic20  -11.201   2.5249
4031          cashout      0.022692      1.455179  Topic20  -11.201   2.5250
2674              ssn      0.022692      1.454795  Topic20  -11.201   2.5252

[765 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
763      12  0.978429         #
1262     11  0.999237      able
1119      6  0.993839  absolute
501      16  0.998647    access
232       5  0.999938   account
...     ...       ...       ...
86       16  0.996426     worry
707      10  0.999085     worth
136       1  0.999939     write
495       2  0.999488      year
824      14  0.992148        zu

[457 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 16, 14, 18, 2, 7, 4, 17, 15, 6, 8, 20, 11, 13, 5, 1, 9, 3, 19, 10])

In [29]:
# Tf-Idf
pyLDAvis.gensim.prepare(lda_model_tfidf, tfidf_corpus, id2word)

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.444829 -0.175297       1        1  39.655268
3     -0.189550  0.365623       2        1  19.136069
12    -0.059961  0.010210       3        1  12.895659
15     0.018245 -0.014750       4        1   5.746884
13     0.029115 -0.015822       5        1   5.599424
11     0.040629 -0.015587       6        1   3.365973
4      0.038435 -0.015775       7        1   2.983341
0      0.044905 -0.014060       8        1   2.870918
16     0.044425 -0.014430       9        1   1.973143
5      0.045451 -0.013258      10        1   1.641053
7      0.045474 -0.012574      11        1   1.160464
10     0.044961 -0.011319      12        1   0.741038
1      0.044813 -0.011107      13        1   0.618312
17     0.044019 -0.010149      14        1   0.475175
19     0.043042 -0.009215      15        1   0.446487
8      0.043166 -0.009328      16        1   0.292952
14     0.042937 -0.009123      17        1   0.189411
2      0.042487 -0.008736      18        1   0.167827
18     0.041118 -0.007651      19        1   0.023762
9      0.041118 -0.007651      20        1   0.016840, topic_info=             Term         Freq        Total Category  logprob  loglift
334         store  2093.000000  2093.000000  Default  30.0000  30.0000
924          bank   956.000000   956.000000  Default  29.0000  29.0000
175       profile  1836.000000  1836.000000  Default  28.0000  28.0000
92          guide  1749.000000  1749.000000  Default  27.0000  27.0000
58         market  3076.000000  3076.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
21408   helps.two     0.000435     0.730651  Topic20 -11.0709   1.2636
21407    days.and     0.000435     0.730651  Topic20 -11.0709   1.2636
21406    provence     0.000435     0.730651  Topic20 -11.0709   1.2636
21405  l'occitane     0.000435     0.730651  Topic20 -11.0709   1.2636
21404     here.if     0.000435     0.730651  Topic20 -11.0709   1.2636

[3390 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1262      6  0.993943        able
501       6  0.996070      access
232       3  0.999667     account
100       2  0.996013         add
307       6  0.971947  additional
...     ...       ...         ...
707      10  0.976284       worth
136       2  0.999428       write
187       5  0.996985       wrong
540       9  0.998936         wsm
495       3  0.996003        year

[299 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 13, 16, 14, 12, 5, 1, 17, 6, 8, 11, 2, 18, 20, 9, 15, 3, 19, 10])

# 6) Hyperparameter Optimization - 

1) **Alpha** controls the number of topics expected in the document. A low value of ‘α’ implies a fewer number of topics and a higher value implies a higher number topics in the mix. 

2) **Beta** controls the distribution of words per topic. At lower values of ‘β’, the topics will likely have fewer words and at higher values topics will likely have more words.

3) If the same keywords being repeated in multiple topics, it’s probably a sign that the ‘k’ is too large.

In [46]:
def compute_coherence_values(corpus, id2word, k, alpha, beta, texts):
    '''
    Compute c_v coherence for a range of number of topics.
    '''
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=k, id2word=id2word, random_state=3, update_every=1, chunksize=100, passes=10, alpha=alpha, eta=beta)
    coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    return coherence_model.get_coherence()

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
# Function inputs
corpus = bow_corpus
id2word = id2word
texts = docs_lemmatized
file_name = 'bow_lda_tuning_results.csv'

grid = {}
grid['validation_set'] = {}

# Topic range
min_topics = 3
max_topics =20
step_size = 2
num_topics_range = list(range(min_topics, max_topics, step_size))

# Alpha = document-topic density
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta = topic-word density
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

for k in num_topics_range:
    for a in alpha:
        for b in beta:
            cv = compute_coherence_values(corpus, id2word, k, a, b, texts)
            
            model_results['Topics'].append(k)
            model_results['Alpha'].append(a)
            model_results['Beta'].append(b)
            model_results['Coherence'].append(cv)

pd.DataFrame(model_results).to_csv('data/'+file_name, index= False)

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyboardInterrupt: 

In [45]:
print(num_topics_range)

[3, 5, 7, 9, 11, 13, 15, 17, 19]


C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
